In [294]:
import pandas as pd
#import seaborn as sns

import plotly.graph_objects as go
#from plotly.subplots import make_subplots

from scipy import stats
from datetime import datetime

Obtaining the datasets and adding the average row for the gas dataset

In [295]:
meralco_df = pd.read_csv('MonthlyPumpPrices.csv', parse_dates=['Date'])
ron95_data = meralco_df[meralco_df['Product'] == "Gasoline (RON95)"].sort_values('Date')
ron95_data = ron95_data[(ron95_data['Date'] >= '2020-01-01') & 
                       (ron95_data['Date'] <= '2025-01-01')]
ron95_data['Average Price'] = (ron95_data['Overall Range Min'] + ron95_data['Overall Range Max']) / 2


Monthly gas prices graph: (from 2020 to 2025)

In [296]:
# Color definitions
dry_season_color = 'rgba(255, 165, 0, 0.2)'  # Orange
wet_season_color = 'rgba(0, 191, 255, 0.2)'   # Light Blue
price_line_color = '#DC143C'                   # Crimson
price_range_color = 'rgba(158, 27, 50, 0.5)'  # Dark Red with transparency

# Create figure
fig = go.Figure()

# Add seasonal background shapes first (so they appear behind the data)
for year in range(2019, 2025):
    # Dry season (Dec-May)
    fig.add_vrect(
        x0=max(datetime(year, 12, 1), datetime(2020, 1, 1)),
        x1=min(datetime(year+1, 6, 1), datetime(2024, 12, 31)),
        fillcolor=dry_season_color,
        layer="below",
        line_width=0
    )
    
    # Wet season (Jun-Nov)
    fig.add_vrect(
        x0=max(datetime(year, 6, 1), datetime(2020, 1, 1)),
        x1=min(datetime(year, 12, 1), datetime(2024, 12, 31)),
        fillcolor=wet_season_color,
        layer="below",
        line_width=0
    )

# Add price range (fill between)
fig.add_trace(go.Scatter(
    x=ron95_data['Date'],
    y=ron95_data['Overall Range Max'],
    fill=None,
    mode='lines',
    line=dict(width=0),
    showlegend=False,
    hoverinfo='skip'
))

fig.add_trace(go.Scatter(
    x=ron95_data['Date'],
    y=ron95_data['Overall Range Min'],
    fill='tonexty',
    mode='lines',
    fillcolor=price_range_color,
    line=dict(width=0),
    name='Price Range',
    hovertemplate="<b>%{x|%b %Y}</b><br>Range: %{y:.2f}–%{text} pesos<extra></extra>",
    text=ron95_data['Overall Range Max'].round(2)
))

# Add average price line
fig.add_trace(go.Scatter(
    x=ron95_data['Date'],
    y=ron95_data['Average Price'],
    mode='lines+markers',
    line=dict(color=price_line_color, width=3),
    marker=dict(size=8),
    name='Average Price',
    hovertemplate="<b>%{x|%b %Y}</b><br>Avg: %{y:.2f} pesos<extra></extra>"
))

# Layout customization
fig.update_layout(
    title={
        'text': "<b>Cost per Liter for Gasoline (RON95) Monthly Price Trends (2020-2024)</b>",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'font': dict(size=18, color='#333333')
    },
    xaxis=dict(
        title="Date",
        range=['2020-01-01', '2024-12-31'],
        tickformat="%Y",
        dtick="M12",  # Yearly ticks
        showgrid=False
    ),
    yaxis=dict(
        title="Cost (pesos)",
        showgrid=True,
        gridcolor='rgba(0,0,0,0.1)'
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    plot_bgcolor='#FAFAFA',
    paper_bgcolor='white',
    hovermode="x unified",
    margin=dict(t=100)
)

# Custom legend for seasons (requires adding dummy traces)
fig.add_trace(go.Scatter(
    x=[None],
    y=[None],
    mode='markers',
    marker=dict(size=15, color=dry_season_color),
    name='Dry Season (Dec-May)'
))

fig.add_trace(go.Scatter(
    x=[None],
    y=[None],
    mode='markers',
    marker=dict(size=15, color=wet_season_color),
    name='Wet Season (Jun-Nov)'
))

fig.show()

# Save as HTML
#fig.write_html("gasoline_price_trends_interactive.html")

Monthly electricity prices graph (2020 to 2025)

In [297]:
## Relevant functions


def get_specific_kwh_df(kwh_consumption: int):
    meralco_df = pd.read_csv('MeralcoSummaryOfRates.csv')
    meralco_df['Date'] = pd.to_datetime(meralco_df['Year'].astype(str) + '-' + meralco_df['Month'].astype(str), format='%Y-%m')

    # Reorder the columns to make 'Date' the first column
    cols = ['Date'] + [col for col in meralco_df.columns if col != 'Date']
    meralco_df = meralco_df[cols]

    chosen_kwh_data = meralco_df[(meralco_df['kWh Consumption'] == kwh_consumption) & (meralco_df['For Lifeline Customers'] == False)]
    chosen_kwh_data = chosen_kwh_data[(chosen_kwh_data['Date'].dt.year >= 2020) & (chosen_kwh_data['Date'].dt.year <= 2024)]

    return chosen_kwh_data

def plot_specific_kwh_df(kwh_consumption: int = 200) -> None:
    df_to_plot = get_specific_kwh_df(kwh_consumption)

    # Color definitions
    dry_season_color = 'rgba(204, 102, 0, 0.3)'  # #CC6600 with alpha
    wet_season_color = 'rgba(102, 153, 102, 0.3)' # #669966 with alpha
    line_color = 'rgba(0, 0, 255, 1)'            # Blue

    # Create figure
    fig = go.Figure()

    # Add seasonal background shapes first (so they appear behind the data)
    for year in range(2019, 2025):
        # Dry season (Dec-May)
        fig.add_vrect(
            x0=max(datetime(year, 12, 1), datetime(2020, 1, 1)),
            x1=min(datetime(year+1, 6, 1), datetime(2024, 12, 31)),
            fillcolor=dry_season_color,
            layer="below",
            line_width=0,
        )
        
        # Wet season (Jun-Nov)
        fig.add_vrect(
            x0=max(datetime(year, 6, 1), datetime(2020, 1, 1)),
            x1=min(datetime(year, 12, 1), datetime(2024, 12, 31)),
            fillcolor=wet_season_color,
            layer="below",
            line_width=0,
        )

    # Add main line plot
    fig.add_trace(go.Scatter(
        x=df_to_plot['Date'],
        y=df_to_plot['Total Bill'],
        mode='lines+markers',
        line=dict(color=line_color, width=2),
        marker=dict(size=8),
        name='Electricity Cost',
        hovertemplate="<b>%{x|%b %Y}</b><br>Cost: %{y:.2f} pesos<extra></extra>"
    ))

    # Layout customization
    fig.update_layout(
        title={
            'text': f"<b>Meralco Cost per kW for {kwh_consumption} kWh Consumption (2020-2024, Non-Lifeline)</b>",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'font': dict(size=18, color='#333333')
        },
        xaxis=dict(
            title="Date",
            range=['2020-01-01', '2024-12-31'],
            tickformat="%Y",
            dtick="M12",  # Yearly ticks
            showgrid=False
        ),
        yaxis=dict(
            title="Cost (pesos)",
            showgrid=True,
            gridcolor='rgba(0,0,0,0.1)'
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        plot_bgcolor='#FAFAFA',
        paper_bgcolor='white',
        hovermode="x unified"
    )
    fig.add_trace(go.Scatter(
    x=[None],
    y=[None],
    mode='markers',
    marker=dict(size=15, color=dry_season_color),
    name='Dry Season (Dec-May)'
    ))

    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(size=15, color=wet_season_color),
        name='Wet Season (Jun-Nov)'
    ))

    fig.show()

    # Optional: Save as HTML
    # fig.write_html(f"electricity_cost_{kwh_consumption}kWh.html")


In [298]:
plot_specific_kwh_df(200)

Seasonality for raw gas and electricity (Just curious)

In [299]:
res = []
chosen_kwh_data = get_specific_kwh_df(200)
for i in chosen_kwh_data['Year'].unique():
    res.append(chosen_kwh_data[chosen_kwh_data['Year'] == i]['Total Bill'].values)

print(f"Electricity, Yearly seasonality: {stats.kruskal(*res)}")

res = []
for i in ron95_data['Date'].dt.year.unique():
    year = ron95_data['Date'].dt.year
    res.append(ron95_data[year == i]['Average Price'].values)

print(f"Gas, Yearly seasonality: {stats.kruskal(*res)}")

res = []
for i in chosen_kwh_data['Month'].unique():
    res.append(chosen_kwh_data[chosen_kwh_data['Month'] == i]['Total Bill'].values)

print(f"Electricity, Monthly seasonality: {stats.kruskal(*res)}")

res = []
for i in ron95_data['Date'].dt.month.unique():
    month = ron95_data['Date'].dt.month
    res.append(ron95_data[month == i]['Average Price'].values)

print(f"Gas, Monthly seasonality: {stats.kruskal(*res)}")

Electricity, Yearly seasonality: KruskalResult(statistic=48.66885245901639, pvalue=6.845482130987131e-10)
Gas, Yearly seasonality: KruskalResult(statistic=46.01031200423054, pvalue=9.038308313091285e-09)
Electricity, Monthly seasonality: KruskalResult(statistic=1.1455737704918079, pvalue=0.9998998777588135)
Gas, Monthly seasonality: KruskalResult(statistic=1.5203595980962632, pvalue=0.9995937999261401)


Comparing gas and electricity

In [300]:
def make_comparison_dataframe(km_l: float, km_kwh: float, kwh_consumption: int = 200):
    df_gas = ron95_data.copy()
    df_elec = get_specific_kwh_df(kwh_consumption)

    #print(df_elec)

    # Drop excess month in df_gas
    date_to_drop = pd.to_datetime('2025-01-01')
    df_gas = df_gas[df_gas['Date'] != date_to_drop]

    monthly_date_range = pd.date_range(start='2020-01-01', end='2024-12-31', freq='MS')
    # Create a new DataFrame with the monthly date range
    df_output = pd.DataFrame({'Date': monthly_date_range.date,
                              'Gas cost/km':None,
                              'Electricity cost/km':None,
                              'Difference': None})
    
    df_gas['Date'] = pd.to_datetime(df_gas['Date'])
    df_elec['Date'] = pd.to_datetime(df_elec['Date'])
    df_output['Date'] = pd.to_datetime(df_output['Date'])

    df_gas = df_gas.set_index('Date')
    df_elec = df_elec.set_index('Date')
    df_output = df_output.set_index('Date')

    # Calculate cost per km for each row
    for date in df_output.index:
        gas_price = df_gas.loc[date, 'Average Price']
        df_output.loc[date, 'Gas cost/km'] = gas_price / km_l  # PHP per km

        elec_price = df_elec.loc[date, 'Total Bill']
        df_output.loc[date, 'Electricity cost/km'] = elec_price / km_kwh  # PHP per km

    # Calculate the difference (Gas cost - Electricity cost)
    df_output['Difference'] = df_output['Gas cost/km'] - df_output['Electricity cost/km']

    df_output = df_output.reset_index()
    return df_output

# toyota vios (15) VS byd ATTO 3 (6.25)
"""
test data
30, 40, 50
6.25, 7, 7.5
"""

'\ntest data\n30, 40, 50\n6.25, 7, 7.5\n'

Plotting the dataframe comparison

In [301]:
def plot_comparison_dataframe(km_l: float, km_kwh: float, kwh_consumption: int = 200):
    df = make_comparison_dataframe(km_l, km_kwh, kwh_consumption)

    # Color definitions
    dry_season_color = 'rgba(204, 102, 0, 0.3)'  # #CC6600 with alpha
    wet_season_color = 'rgba(102, 153, 102, 0.3)' # #669966 with alpha
    gas_color = '#EF553B'  # Red
    electricity_color = '#636EFA'  # Blue

    # Create figure
    fig = go.Figure()

    # Add seasonal background shapes first (so they appear behind the data)
    for year in range(2019, 2025):
        # Dry season (Dec-May)
        fig.add_vrect(
            x0=max(datetime(year, 12, 1), df['Date'].min()),
            x1=min(datetime(year+1, 6, 1), df['Date'].max()),
            fillcolor=dry_season_color,
            layer="below",
            line_width=0,
        )
        
        # Wet season (Jun-Nov)
        fig.add_vrect(
            x0=max(datetime(year, 6, 1), df['Date'].min()),
            x1=min(datetime(year, 12, 1), df['Date'].max()),
            fillcolor=wet_season_color,
            layer="below",
            line_width=0,
        )

    # Add Gas line plot
    fig.add_trace(go.Scatter(
        x=df['Date'],
        y=df['Gas cost/km'],
        mode='lines+markers',
        line=dict(color=gas_color, width=2),
        marker=dict(size=8),
        name='Gas Cost/km',
        hovertemplate="<b>%{x|%b %Y}</b><br>Gas Cost: PHP %{y:.3f}<extra></extra>"
    ))

    # Add Electricity line plot
    fig.add_trace(go.Scatter(
        x=df['Date'],
        y=df['Electricity cost/km'],
        mode='lines+markers',
        line=dict(color=electricity_color, width=2),
        marker=dict(size=8),
        name='Electricity Cost/km',
        hovertemplate="<b>%{x|%b %Y}</b><br>Electricity Cost: PHP %{y:.3f}<extra></extra>"
    ))

    # Layout customization (matching first plot's style)
    fig.update_layout(
        title={
            'text': "<b>Gas vs. Electricity Cost per Kilometer (2020-2025)</b>",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'font': dict(size=18, color='#333333')
        },
        xaxis=dict(
            title="Date",
            range=[df['Date'].min(), df['Date'].max()],
            tickformat="%Y",
            dtick="M12",  # Yearly ticks
            showgrid=False
        ),
        yaxis=dict(
            title="Cost per km (PHP)",
            showgrid=True,
            gridcolor='rgba(0,0,0,0.1)'
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        plot_bgcolor='#FAFAFA',
        paper_bgcolor='white',
        hovermode="x unified"
    )

    # Add seasonal legend entries (as in first plot)
    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(size=15, color=dry_season_color),
        name='Dry Season (Dec-May)'
    ))

    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(size=15, color=wet_season_color),
        name='Wet Season (Jun-Nov)'
    ))

    fig.add_annotation(
        x=0.02,
        y=-0.15,
        xref="paper",
        yref="paper",
        align="left",
        text=f"<i>Gas cost calculated with {km_l} km/L<br>Electricity cost calculated with {km_kwh} km/kWh and {kwh_consumption} kWh consumption</i>",
        showarrow=False,
        font=dict(size=11, color="#666666")
    )

    # Show the plot
    fig.show()

In [302]:
plot_comparison_dataframe(40, 7, 200)

Statistical Test

In [305]:
res = []
df = make_comparison_dataframe(40, 7, 200)

for i in df['Date'].dt.month.unique():
    time = df['Date'].dt.month
    res.append(df[time == i]['Difference'].values)

print(f"Gas, Monthly seasonality: {stats.kruskal(*res)}")

Gas, Monthly seasonality: KruskalResult(statistic=3.018360655737723, pvalue=0.9904810075505583)


Nutshell plot